In [12]:
import pandas as pd
import os
import datetime as dt
import re
import xlsxwriter
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px

In [13]:
# Загружаем предобработанный отчет (из OP_report)
op_report = pd.read_excel('Сводная направленные от ОП 4.1.2021-10.1.2021.xlsx', 
                          sheet_name="Данные", 
                          engine='openpyxl',
                          keep_default_na=True, 
                          convert_float=True, 
                          dtype={"Ресторан № (собеседование)": str, "Ресторан № (работа)": str,
                                 "ТелефонвформатеОРИС": str}, na_values=[pd.NaT])

In [14]:
op_report.columns

Index(['Рекрутер', 'ФИО', 'ТелефонвформатеОРИС', 'Возраст', 'Гражданство',
       'Источник', 'Телефонное интервью (число)', 'Ресторан № (собеседование)',
       'Ресторан № (работа)', 'Дата собеседования', 'Unnamed: 10',
       'Дошёл/Не дошёл ДА/НЕТ', 'Обратная связь от ресторана',
       'Направление в ОРИС ДА/НЕТ', 'Обратная связь от кандидата',
       'Направлен на оформление ДА/НЕТ', 'Оформился ДА/НЕТ',
       'Прошел МО ДА/НЕТ', 'Вышел в ресторан', 'Смена 4 чаcа ДА/НЕТ',
       'Дата  1 смены ', 'код ресторана', 'Дивизион', 'регион', 'город'],
      dtype='object')

In [15]:
def add_status_columns(op_report):
    '''
    Проставляет статусы для направленных кандидатов на основе информации, заполненной рекруторами
    '''
    op_report["Отработал 1-ю смену"] = (~op_report['Дата  1 смены '].isna())
    op_report["На оформлении"] = ((op_report['Направлен на оформление ДА/НЕТ'] == 'ДА') | \
                                  ((op_report['Обратная связь от ресторана'].str.contains("отправлен на оформление")))) & \
                                    (op_report['Дошёл/Не дошёл ДА/НЕТ'] != "нет") \
                                    & (op_report['Направлен на оформление ДА/НЕТ'] != "НЕТ") \
                                    & (op_report['Оформился ДА/НЕТ'] != "НЕТ") & (op_report['Прошел МО ДА/НЕТ'] != "нет") \
                                    & (op_report['Вышел в ресторан'] != "НЕТ") & (~op_report["Отработал 1-ю смену"])
    op_report["Отказ по любым причинам"] = (op_report['Направлен на оформление ДА/НЕТ'] == 'НЕТ') | \
                                            (op_report['Оформился ДА/НЕТ'] == "НЕТ") | \
                                            (op_report['Прошел МО ДА/НЕТ'] == "НЕТ") \
                                            | (op_report['Вышел в ресторан'] == "НЕТ") | \
                                            (op_report['Обратная связь от ресторана'].str.contains("отказ")) | \
                                            (op_report['Обратная связь от кандидата'].str.contains("отказ")) | \
                                            (op_report['Дошёл/Не дошёл ДА/НЕТ'] == "нет") 
    op_report["Ждем ОС"] = ((op_report['Дошёл/Не дошёл ДА/НЕТ'] == "да" ) | \
                            (op_report['Обратная связь от ресторана'].isna()) | (op_report['Обратная связь от кандидата'].isna())) & \
                            (op_report['Дата собеседования'] < pd.to_datetime(get_this_monday())) \
                            & (~op_report["Отказ по любым причинам"]) \
                            & (~op_report["На оформлении"]) & (~op_report["Отработал 1-ю смену"])
    op_report["Направлен на собеседование"] = (op_report['Дата собеседования'] >= pd.to_datetime(get_this_monday())) & \
                                                (~op_report["На оформлении"]) \
                                                & (~op_report["Отказ по любым причинам"]) & (~op_report["Отработал 1-ю смену"])
    op_report["Статус не определен"] = (~op_report["На оформлении"]) & (~op_report["Отказ по любым причинам"]) & \
                                        (~op_report["Ждем ОС"]) & (~op_report["Направлен на собеседование"]) & (~op_report["Отработал 1-ю смену"])
    return op_report


def select_period(report, month, year):
    """
    Отбирает данные за нужный год и месяц
    """
    report = report[(report['Дата собеседования'].dt.month == month) & (report['Дата собеседования'].dt.year == year)]
    return report

def get_this_monday():
    """Находит дату последнего понеделтника"""
    today = dt.date.today()
    this_monday = today - dt.timedelta(days=today.weekday())
    return this_monday

In [16]:
# Проставляем статусы
report_with_status = add_status_columns(op_report)
# Выбирает данные за исследуемый месяц
dec = select_period(report_with_status, 12, 2020)

In [18]:
def group_table(report):
    """
    Возвращает сгруппированную по регионам таблицу со статусами кандидатов
    """
    report = report.copy()
    groupped_report = report.groupby(["регион"])[["На оформлении", "Отказ по любым причинам", "Ждем ОС", 
                                                  "Направлен на собеседование", 
                                "Статус не определен", "Отработал 1-ю смену"]].sum().reset_index()
    return groupped_report

final_table = group_table(dec)

In [21]:
final_table.describe()

,На оформлении,Отказ по любым причинам,Ждем ОС,Направлен на собеседование,Статус не определен,Отработал 1-ю смену
count,17.000000,17.000000,17.000000,17.0,17.0,17.000000
mean,12.941176,92.117647,11.352941,0.0,0.0,1.882353
std,8.257350,56.960384,6.604366,0.0,0.0,1.798692
min,3.000000,26.000000,4.000000,0.0,0.0,0.000000
25%,6.000000,47.000000,6.000000,0.0,0.0,0.000000
50%,11.000000,84.000000,9.000000,0.0,0.0,2.000000
75%,18.000000,115.000000,14.000000,0.0,0.0,3.000000
max,32.000000,231.000000,28.000000,0.0,0.0,5.000000


In [20]:
fig = px.bar(final_table, x='регион', y=["На оформлении", "Отказ по любым причинам", "Отработал 1-ю смену", "Ждем ОС", 
                                                     "Направлен на собеседование", "Статус не определен"], 
             title="Статус кандидатов по дням")
fig.show()